# TEO Simulation in PyBullet

This notebook contains an example using pybullet to perform a stability test. We will first load the robot in its initial position, enable position control and simulate for a few seconds to correct for the ability of the robot to mantain the position. Then a joint/torque sensor will be aded to the ankle to measeure the ground reaction as it performs a movement.

## Setup

Import the necesary libraries

In [1]:
import pybullet as p
import numpy as np
import pybullet_data
import time
from math import radians, degrees
import os, inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
urdf_root = os.path.join(parentdir,"models")

Start the physics client in Graphic mode

In [2]:
physicsClient = p.connect(p.GUI)

Add path for PyBullet exaples and useful assets such as the ground plane 

In [3]:
p.setAdditionalSearchPath(pybullet_data.getDataPath())

Set the gravity

In [4]:
p.setGravity(0,0,-9.79983)

Load the ground

In [5]:
planeId = p.loadURDF("plane.urdf")

Load the TEO urdf file above the ground plane

In [6]:
teoStartOrientation = p.getQuaternionFromEuler([0,0,0])
teoStartPosition = [0,0,0.855]
urdf_path = os.path.join(urdf_root,"TEO.urdf")
teoId = p.loadURDF(urdf_path,teoStartPosition ,teoStartOrientation)

If loaded correctly we should see all the joint names and their joint ids

In [7]:
n_joints = p.getNumJoints(teoId)
name2id={}
for i in range(n_joints):
    name2id[p.getJointInfo(teoId,i)[1]]=i
    print(p.getJointInfo(teoId,i)[1])

b'waist_yaw'
b'waist_pitch'
b'head_yaw'
b'head_pitch'
b'l_shoulder_pitch'
b'l_shoulder_roll'
b'l_shoulder_yaw'
b'l_elbow_pitch'
b'l_wrist_yaw'
b'l_wrist_pitch'
b'r_shoulder_pitch'
b'r_shoulder_roll'
b'r_shoulder_yaw'
b'r_elbow_pitch'
b'r_wrist_yaw'
b'r_wrist_pitch'
b'l_hip_yaw'
b'l_hip_roll'
b'l_hip_pitch'
b'l_knee_pitch'
b'l_ankle_pitch'
b'l_ankle_roll'
b'l_sole_joint'
b'r_hip_yaw'
b'r_hip_roll'
b'r_hip_pitch'
b'r_knee_pitch'
b'r_ankle_pitch'
b'r_ankle_roll'
b'r_sole_joint'


### Define starting position

We set the desired joint positions for the joint controllers to achieve. We also set the max force to the ones defined by the urdf and the max velocities to what we desire.

In [46]:
n_joints = p.getNumJoints(teoId)

jointIndices = [0] * n_joints
targetPos = [0.0] * n_joints
maxVelocities = [radians(1)] * n_joints
maxForces = [0.0] * n_joints


for i in range(n_joints):
    jointIndices[i] = i
    maxForces[i] = p.getJointInfo(teoId,i)[10]
    
    
    if p.getJointInfo(teoId,i)[1] == 'waist_yaw':
        targetPos[i] = 0
    elif p.getJointInfo(teoId,i)[1] == 'r_shoulder_roll':
        targetPos[i] = radians(-0)
        
    elif p.getJointInfo(teoId,i)[1] == 'r_hip_roll':
        targetPos[i] = radians(0)
    elif p.getJointInfo(teoId,i)[1] == 'r_hip_pitch':
        targetPos[i] = radians(-0)
    elif p.getJointInfo(teoId,i)[1] == 'r_knee_pitch':
        targetPos[i] = radians(0)
    elif p.getJointInfo(teoId,i)[1] == 'r_ankle_pitch':
        targetPos[i] = radians(-0)
    elif p.getJointInfo(teoId,i)[1] == 'r_ankle_roll':
        targetPos[i] = radians(-0)
        
    elif p.getJointInfo(teoId,i)[1] == 'l_hip_roll':
        targetPos[i] = radians(0)
    elif p.getJointInfo(teoId,i)[1] == 'l_ankle_roll':
        targetPos[i] = radians(-0)
        
    elif p.getJointInfo(teoId,i)[1] == 'l_shoulder_roll':
        targetPos[i] = radians(0)
    else:
        targetPos[i] = 0

jointIndices.pop(name2id[b'r_sole_joint'])
jointIndices.pop(name2id[b'l_sole_joint'])

targetPos.pop(name2id[b'r_sole_joint'])
targetPos.pop(name2id[b'l_sole_joint'])

maxVelocities.pop(name2id[b'r_sole_joint'])
maxVelocities.pop(name2id[b'l_sole_joint'])

maxForces.pop(name2id[b'r_sole_joint'])
maxForces.pop(name2id[b'l_sole_joint'])


0.0

We set the joint control with the created arrays and set it to position control.

In [47]:
mode = p.POSITION_CONTROL

p.setJointMotorControlArray(teoId,
                            jointIndices, 
                            controlMode=mode,
                            forces=maxForces,
                            targetVelocities = maxVelocities,
                            targetPositions = targetPos
                           )

Advance simulation until the robot is stable

## Gait

#### Foot position

In [22]:
timestep = 1/240
p.setTimeStep(timestep)

In [11]:
f = interpol(0,0,0,1,0,0)

In [12]:
import matplotlib.pyplot as plt
import numpy as np

In [67]:
gait_step_time=5
step_height=0.01
step_length=0
step_width=0
ori_x=0
ori_y=0
ori_z=0
ori_quat=p.getLinkState(teoId,name2id[b'r_sole_joint'])[1]
def interpol(xi,xti,xtti,xf,xtf,xttf):
    T = gait_step_time/2
    return lambda t : (6*t**5*xf)/T**5 - (15*t**4*xf)/T**4 + (10*t**3*xf)/T**3 + xi - (6*t**5*xi)/T**5 + (15*t**4*xi)/T**4 - (10*t**3*xi)/T**3 - (3*t**5*xtf)/T**4 + (7*t**4*xtf)/T**3 - (4*t**3*xtf)/T**2 + t*xti - (3*t**5*xti)/T**4 + (8*t**4*xti)/T**3 - (6*t**3*xti)/T**2 + (t**5*xttf)/(2.*T**3) - (t**4*xttf)/T**2 + (t**3*xttf)/(2.*T) + (t**2*xtti)/2. - (t**5*xtti)/(2.*T**3) + (3*t**4*xtti)/(2.*T**2) - (3*t**3*xtti)/(2.*T)

In [68]:
pos = p.getLinkState(teoId,name2id[b'r_sole_joint'])[0]
phix_1=interpol(pos[0],0,0,pos[0]+step_length/2,0,0)
phix_2=interpol(pos[0]+step_length/2,0,0,pos[0]+step_length,0,0)
phiy_1=lambda t : pos[1]
phiy_2=lambda t : pos[1]
phiz_1=interpol(0,0,pos[2],0,0,pos[2]+step_height)
phiz_2=interpol(0,0,pos[2]+step_height,0,0,pos[2])

In [69]:
# setup simulation
p.resetSimulation()
planeId = p.loadURDF("plane.urdf")
p.setGravity(0,0,-9.79983)
teoStartOrientation = p.getQuaternionFromEuler([0,0,0])
teoStartPosition = [0,0,0.855]
teoId = p.loadURDF(urdf_path,teoStartPosition ,teoStartOrientation)
mode = p.POSITION_CONTROL

#end-effector id
ef = name2id[b'r_sole_joint']

# Setup interpolators
pos = p.getLinkState(teoId,ef)[0]
print(pos)
phix_1=interpol(pos[0],0,0,pos[0]+step_length/2,0,0)
phix_2=interpol(pos[0]+step_length/2,0,0,pos[0]+step_length,0,0)
phiy_1=lambda t : pos[1]
phiy_2=lambda t : pos[1]
phiz_1=interpol(0,0,pos[2],0,0,pos[2]+step_height)
phiz_2=interpol(0,0,pos[2]+step_height,0,0,pos[2])

time.sleep(1)


for i in range(int(gait_step_time*1.5/timestep)):
    if i < (half_step_time/timestep):
        targetPos = list(p.calculateInverseKinematics(teoId, ef,[phix_1(i),phiy_1(i),phiz_1(i)],ori_quat))
        p.setJointMotorControlArray(teoId,
                            jointIndices, 
                            controlMode=mode,
                            forces=maxForces,
                            targetVelocities = maxVelocities,
                            targetPositions = targetPos
                           )
    elif i < (2*half_step_time/timestep):
        targetPos = list(p.calculateInverseKinematics(teoId, ef,[phix_2(i),phiy_2(i),phiz_2(i)],ori_quat))
        p.setJointMotorControlArray(teoId,
                    jointIndices, 
                    controlMode=mode,
                    forces=maxForces,
                    targetVelocities = maxVelocities,
                    targetPositions = targetPos
                   )
    else:
        targetPos = list(p.calculateInverseKinematics(teoId, ef,[phix_2(2*half_step_time/timestep),phiy_2(2*half_step_time/timestep),phiz_2(2*half_step_time/timestep)],ori_quat))
        p.setJointMotorControlArray(teoId,
                    jointIndices, 
                    controlMode=mode,
                    forces=maxForces,
                    targetVelocities = maxVelocities,
                    targetPositions = targetPos
                   )   
    p.stepSimulation()
    time.sleep(1./240.)

(0.022629, -0.126052, 0.0203249999999999)


In [39]:
len(jointIndices)

30

In [27]:
phix_1(0.2)

0.022651626304

In [31]:
(0.0,
 0.010153074920391499,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0)

0.010153074920391499

In [77]:
# setup simulation
p.resetSimulation()
planeId = p.loadURDF("plane.urdf")
p.setGravity(0,0,-9.79983)
teoStartOrientation = p.getQuaternionFromEuler([0,0,0])
teoStartPosition = [0,0,0.855]
teoId = p.loadURDF(urdf_path,teoStartPosition ,teoStartOrientation)
mode = p.POSITION_CONTROL

#end-effector id
ef = name2id[b'r_wrist_pitch']
pos = p.getLinkState(teoId,ef)[0]
targetPos = list(p.calculateInverseKinematics(teoId, ef,pos,ori_quat))

mode = p.VELOCITY_CONTROL
        
p.setJointMotorControlArray(teoId,
                    jointIndices, 
                    controlMode=mode,
                    forces=maxForces,
                    targetVelocities = maxVelocities,
                    targetPositions = targetPos)

for i in range(1000):
    p.stepSimulation()
    time.sleep(timestep)